In [1]:
import pandas as pd

In [25]:
movies = pd.read_csv('raw_data/movies.csv')
actors = pd.read_csv('raw_data/actors.csv')
crew = pd.read_csv('raw_data/crew.csv')
languages = pd.read_csv('raw_data/languages.csv')
genres = pd.read_csv('raw_data/genres.csv')
studios = pd.read_csv('raw_data/studios.csv')
# themes = pd.read_csv('raw_data/themes.csv')
# releases = pd.read_csv('raw_data/releases.csv')
countries = pd.read_csv('raw_data/countries.csv')

In [26]:
movies = movies.drop(columns='tagline', axis=1)

In [27]:
actors = actors.drop(columns='role', axis=1)
actors = actors.dropna()

In [18]:
# crew.isnull().sum()
# crew.role.unique()

id      0
role    0
name    1
dtype: int64

In [28]:
# movies.head()
# id = 1000001
# temp = crew[crew['id']==1000001]
crew = crew[crew['role'].isin(['Director', 'Producer', 'Writer', 'Cinematography', 'Composer'])] #'Songs'
#consider relabel those with small proportion to new category "Others"
#embeddings make sense, but we will have little control
#TODO: we check later how we deal with this feature!

In [41]:
languages = languages[languages['type'].isin(['Language', 'Primary language'])].drop(columns='type')

In [7]:
# genres.genre.unique() #OneHotEncoder

In [ ]:
# studios.studio.value_counts()/studios.studio.value_counts().sum()
#consider relabel those with small proportion to new category "Others" or drop them
#embeddings make sense, but we will have little control
#TODO: we check later how we deal with this feature!

studio
BBC                      0.005241
Warner Bros. Pictures    0.004743
Columbia Pictures        0.004416
ARTE                     0.004157
Metro-Goldwyn-Mayer      0.004106
                           ...   
Island Creek Pictures    0.000001
Extremo Sur Films        0.000001
KHF Media                0.000001
Kude Media               0.000001
Cineploit Pictures       0.000001
Name: count, Length: 160881, dtype: float64

In [ ]:
# countries # categorical, high-cardinality, embeddings
# countries # if we chose studios, this one can be left out due to collinearity

,id,country
0,1000001,UK
1,1000001,USA
2,1000002,South Korea
3,1000003,USA
4,1000004,Germany
...,...,...
693471,1941593,China
693472,1941594,USA
693473,1941595,USA
693474,1941596,China


In [30]:
new_crew = (
    crew.groupby('id')
    .apply(lambda x: x.groupby('role')['name'].apply(list).to_dict())
    .reset_index(name='crew_dict')
)

/tmp/ipykernel_52055/2434692008.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.groupby('role')['name'].apply(list).to_dict())


In [33]:
new_genres = (
    genres.groupby('id')['genre']
    .apply(list)  # Aggregates genres into a list
    .reset_index(name='genre_list')  # Converts to DataFrame and renames the column
)

In [34]:
new_studios = (
    studios.groupby('id')['studio']
    .apply(list)  # Aggregates genres into a list
    .reset_index(name='studio_list')  # Converts to DataFrame and renames the column
)

In [35]:
new_actors = (
    actors.groupby('id')['name']
    .apply(list)  # Aggregates genres into a list
    .reset_index(name='actor_list')  # Converts to DataFrame and renames the column
)

In [43]:
data = movies.merge(new_genres, how='left', on='id').merge(new_actors, how='left', on='id').merge(languages, how='left', on='id').merge(new_studios, how='left', on='id').merge(new_crew, how='left', on='id')

In [47]:
data.to_csv('compiled_movies.csv', index=False)

In [ ]:
genres['genre_list'] = genres.groupby('id')['genre'].apply(lambda x: ', '.join(x)).reset_index()
# movies_with_genres = movies.merge(aggregated_genres, on='id', how='left')
# Convert the genre strings into lists
# genres['genre_list'] = genres['genre'].str.split(', ')
# Handle missing values
movies_with_genres['genre_list'] = movies_with_genres['genre_list'].apply(lambda x: x if isinstance(x, list) else [])
movies_with_genres.drop(columns="genre", axis=1, inplace=True)
unique_genres = set([genre for sublist in movies_with_genres['genre_list'] for genre in sublist])
unique_genres = sorted(list(unique_genres))
print(unique_genres)
for genre in unique_genres:
    movies_with_genres[genre] = movies_with_genres['genre_list'].apply(lambda x: 1 if genre in x else 0)

In [ ]:
data = movies.merge(actors, how='left', on='id').merge()

# SUMMARY
- import csv files
- 

ADVISE: high-cardiality --> may make sense to drop the rare guys or not